In [11]:
from analysis.probability.simple import simple_posneg
from evaluate import evaluate_single

def eval_simple(note):
    return evaluate_single(simple_posneg(note), root_note=note)

In [12]:
import numpy as np
from analysis.musical import consonance_ordered_notes, empirical_probabilities
from analysis.spike_tensor import generate_spike_tensor
from cache import get_spikes
from evaluate import predicted_consonance_scores, predicted_probabilities

root_note = "C4"
notes = consonance_ordered_notes(root_note)
notes_spikes = [get_spikes(note) for note in notes]
consonance_ordered_tensors = [generate_spike_tensor(spikes) for spikes in notes_spikes]

root_tensor = generate_spike_tensor(get_spikes(root_note))
probability_tensor = simple_posneg(root_note)

def try_eval(tensors, metric, debug=False):
    scores = predicted_consonance_scores(probability_tensor, tensors, root_tensor)

    P_predicted = predicted_probabilities(scores)

    if debug:
        def print_arr(arr):
            print(np.round(np.array(arr), 3))
        print("Scores")
        print_arr(scores)
        print("P_predicted")
        print_arr(P_predicted)
        print("P_empirical")
        print_arr(empirical_probabilities())

    print(metric(P_predicted))

In [13]:
from doctest import debug
from evaluate import js_divergence


try_eval(consonance_ordered_tensors, js_divergence, debug=True)

Scores
[-0.82945544 -0.92553053 -0.93005259 -0.93908423 -0.89713392 -0.92324783
 -0.93824591 -0.91400843 -0.92349832 -0.90880194 -0.87044873 -0.90813502]
P_predicted
[0.3033717077339259, 0.03750664961348267, 0.02499292594312205, 0.0, 0.11608753720953043, 0.04382349945104557, 0.0023198662154283157, 0.0693913339973556, 0.04313031261672915, 0.08379906565855662, 0.18993248520107645, 0.08564461635974727]
P_empirical
[0.14666666666666667, 0.13333333333333333, 0.12, 0.10666666666666667, 0.10666666666666667, 0.09333333333333334, 0.08, 0.06666666666666667, 0.05333333333333334, 0.05333333333333334, 0.02666666666666667, 0.013333333333333334]
0.40173398181449904


In [6]:
import numpy as np

def collect_data(metric, N=10):
    data = []
    for i in range(N):
        A = consonance_ordered_tensors[0][0][0]
        np.random.shuffle(consonance_ordered_tensors)
        B = consonance_ordered_tensors[0][0][0]
        if not np.array_equal(A, B):
            value = try_eval(consonance_ordered_tensors, metric)
            data.append(value)

In [7]:
from evaluate import js_divergence


collect_data(js_divergence)

0.47624076106871727
0.38893976009743936
0.3561764366075249
0.4085838509635776
0.3846056120044244
0.37700960002463074
0.4199492153988373
0.37209464946343135
0.29609218702414664
0.33308088483802717


In [ ]:
import os

from cache import SPIKES_DIR


root_note = "C4"
notes = consonance_ordered_notes(root_note)
notes = [notes[0], notes[-1]]
notes_spikes = [get_spikes(note) for note in notes]
tensors = [generate_spike_tensor(spikes) for spikes in notes_spikes]

white_noise_spikes = np.load(os.path.join(SPIKES_DIR, "white_noise.npy"))
tensors.append(generate_spike_tensor(white_noise_spikes))

root_tensor = generate_spike_tensor(get_spikes(root_note))
probability_tensor = simple_posneg(root_note)

scores = predicted_consonance_scores(probability_tensor, tensors, root_tensor)
print("Scores")
print(scores)

P_predicted = predicted_probabilities(scores)
print("P_predicted")
print(P_predicted)



In [2]:
keys = ["C4", "C#4", "D4", "D#4", "E4", "F4", "F#4", "G4", "G#4", "A4", "A#4", "B4"]

for key in keys:
    print(f"{key}: {eval_simple(key)}")

C4: 60
C#4: 57
D4: 64
D#4: 68
E4: 67
F4: 64
F#4: 71
G4: 73
G#4: 71
A4: 75
A#4: 66
B4: 71


In [ ]:
# collect_data(kl_divergence_reverse)
# collect_data(kl_divergence)